###Import necessary packages

We need to import the torch.nn package from pytorch because here we aim to create a Neural Network(1 Hourglass ) by combining various layers  



In [0]:
import torch
from torch import nn

In [0]:
Pool = nn.MaxPool2d

This function returns a layer which  applies batchnormalisation to  a 4D input(x =C from input of size[N,C,H,W]).Batch normalization normalizes the output of a previous activation layer by subtracting the batch mean and dividing by the batch standard deviation across each channel

In [0]:
def batchnorm(x):
    return nn.BatchNorm2d(x.size()[1])(x)

We know the aim of the stacked hour glass is to perform repeated topdown-bottomup inferences.This next neural network module applies a single convolution layer(for bottom-up i.e. extracting spatial features while going from higher reolution to lower resolution) to the input batch,along with relu activation and batchnormalisation layer if needed(specified in the class arguments).This module is used to create other modules like residual modules

inp_dim=number of channels in the input layer
out_dim=number of channels in the output layer


In [0]:
class Conv(nn.Module):
    def __init__(self, inp_dim, out_dim, kernel_size=3, stride = 1, bn = False, relu = True):
        super(Conv, self).__init__()
        self.inp_dim = inp_dim
        self.conv = nn.Conv2d(inp_dim, out_dim, kernel_size, stride, padding=(kernel_size-1)//2, bias=True)
        self.relu = None
        self.bn = None
        if relu:
            self.relu = nn.ReLU()
        if bn:
            self.bn = nn.BatchNorm2d(out_dim)

    def forward(self, x):
        assert x.size()[1] == self.inp_dim, "{} {}".format(x.size()[1], self.inp_dim)
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

1)As explained in the documentation,a residual module is an important part of the hourglass network.One box in the first figure corresponds to one residual module.

2)This network applies (batch-normalisation -> relu ->Convolution layer) 3 times to the input.

3)It also includes a skip connection which just means passing the output of one layer to the one further down.Hence,if inp_dim is not equal to the output dimension we use a skip layer and add the input x to the final output of Residual network.Skip layer is used to make the output from the Residual module equal to have same dime

In [0]:
class Residual(nn.Module):
    def __init__(self, inp_dim, out_dim):
        super(Residual, self).__init__()
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(inp_dim)
        self.conv1 = Conv(inp_dim, int(out_dim/2), 1, relu=False)
        self.bn2 = nn.BatchNorm2d(int(out_dim/2))
        self.conv2 = Conv(int(out_dim/2), int(out_dim/2), 3, relu=False)
        self.bn3 = nn.BatchNorm2d(int(out_dim/2))
        self.conv3 = Conv(int(out_dim/2), out_dim, 1, relu=False)
        self.skip_layer = Conv(inp_dim, out_dim, 1, relu=False)
        if inp_dim == out_dim:
            self.need_skip = False
        else:
            self.need_skip = True
        
    def forward(self, x):
        if self.need_skip:
            residual = self.skip_layer(x)
        else:
            residual = x
        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv3(out)
        out += residual
        return out 


This Hourglass module aims to employ 1 hourglass,with the help of Residual and convolutions modules.Upsampling is done by the 'up2' to a scale factor of 2, i.e it doubles the spatial dimensions of the input image.using the mode nearest neighbour upsampling.

In [1]:
class Hourglass(nn.Module):
    def __init__(self, n, f, bn=None, increase=0):
        super(Hourglass, self).__init__()
        nf = f + increase
        self.up1 = Residual(f, f)
        # Lower branch
        self.pool1 = Pool(2, 2)
        self.low1 = Residual(f, nf)
        self.n = n
        # Recursive hourglass
        if self.n > 1:
            self.low2 = Hourglass(n-1, nf, bn=bn)
        else:
            self.low2 = Residual(nf, nf)
        self.low3 = Residual(nf, f)
        self.up2 = nn.Upsample(scale_factor=2, mode='nearest')

    def forward(self, x):
        up1  = self.up1(x)
        pool1 = self.pool1(x)
        low1 = self.low1(pool1)
        low2 = self.low2(low1)
        low3 = self.low3(low2)
        up2  = self.up2(low3)
        return up1 + up2

NameError: ignored